# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

## Code

In [6]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [7]:
# load outage data
outage = pd.read_excel('outage.xlsx', skiprows=5, usecols=['U.S._STATE', 'POSTAL.CODE', 'CAUSE.CATEGORY', 'OUTAGE.DURATION', 'CUSTOMERS.AFFECTED', 'DEMAND.LOSS.MW', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE', 'NERC'])
outage = outage.drop(index=0)
outage

,U.S._STATE,POSTAL.CODE,CAUSE.CATEGORY,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,RES.PERCEN,COM.PERCEN,IND.PERCEN
1,Minnesota,MN,severe weather,3060,NaN,70000.0,11.6,9.18,6.81,35.549073,32.225029,32.202431
2,Minnesota,MN,intentional attack,1,NaN,NaN,12.12,9.71,6.49,30.032487,34.210389,35.727564
3,Minnesota,MN,severe weather,3000,NaN,70000.0,10.87,8.19,6.07,28.097672,34.501015,37.365983
4,Minnesota,MN,severe weather,2550,NaN,68200.0,11.79,9.25,6.71,31.994099,33.54333,34.439329
5,Minnesota,MN,severe weather,1740,250,250000.0,13.07,10.16,7.74,33.982576,36.20585,29.779498
...,...,...,...,...,...,...,...,...,...,...,...,...
1530,North Dakota,ND,public appeal,720,155,34500.0,8.41,7.8,6.2,37.212544,33.351628,29.435904
1531,North Dakota,ND,fuel supply emergency,NaN,1650,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1532,South Dakota,SD,islanding,59,84,NaN,9.25,7.47,5.53,36.564432,40.124517,23.311051
1533,South Dakota,SD,islanding,181,373,NaN,9.25,7.47,5.53,36.564432,40.124517,23.311051


### Framing the Problem

### Data Cleaning for Outages dataset

In [9]:
#perform probabilistic imputation on multiple columns together
def multi_prob_impute(df, cols):
    missingness = outage[cols[-1]].isna()
    fill_index = np.random.choice(outage[cols[-1]].dropna().index, missingness.sum())
    fill_values = outage.loc[fill_index, cols]
    for col in cols:
        df.loc[missingness, col] = fill_values[col].to_numpy()

# perform probabilistic imputation on single column
def prob_impute(df, col):
    fill_values = np.random.choice(df[col].dropna(), df[col].isna().sum())
    df.loc[df[col].isna(), col] = fill_values
outage = outage.copy()[(outage['CUSTOMERS.AFFECTED'] >= 50000) | (outage['DEMAND.LOSS.MW'] >= 300)]

# use probabilistic imputation to fill missing values in 'RES.PERCEN', 'IND.PERCEN', 'COM.PERCEN'
multi_prob_impute(outage, ['RES.PERCEN', 'IND.PERCEN', 'COM.PERCEN'])
prob_impute(outage, 'RES.PRICE')
prob_impute(outage, 'COM.PRICE')
prob_impute(outage, 'IND.PRICE')

outage['OUTAGE.DURATION'] = outage.groupby('CAUSE.CATEGORY')['OUTAGE.DURATION'].transform(lambda x: x.fillna(x.mean()))
outage['DEMAND.LOSS.MW'] = outage.groupby('CAUSE.CATEGORY')['DEMAND.LOSS.MW'].transform(lambda x: x.fillna(x.mean()))

outage.loc[:, ~outage.columns.isin(['U.S._STATE', 'POSTAL.CODE', 'CAUSE.CATEGORY'])] = outage.loc[:, ~outage.columns.isin(['U.S._STATE', 'POSTAL.CODE', 'CAUSE.CATEGORY'])].astype(float)

outage['RES.MONEY.LOST'] = round(outage['OUTAGE.DURATION']*outage['RES.PERCEN']*outage['DEMAND.LOSS.MW']*outage['RES.PRICE'] / 60000, 2)
outage['COM.MONEY.LOST'] = round(outage['OUTAGE.DURATION']*outage['COM.PERCEN']*outage['DEMAND.LOSS.MW']*outage['COM.PRICE'] / 60000, 2)
outage['IND.MONEY.LOST'] = round(outage['OUTAGE.DURATION']*outage['IND.PERCEN']*outage['DEMAND.LOSS.MW']*outage['IND.PRICE'] / 60000, 2)
outage['TOTAL.MONEY.LOST'] = outage['RES.MONEY.LOST'] + outage['COM.MONEY.LOST'] + outage['IND.MONEY.LOST']
print(outage.head())
print(outage.head().to_markdown())

  U.S._STATE POSTAL.CODE  CAUSE.CATEGORY  OUTAGE.DURATION  DEMAND.LOSS.MW  \
1  Minnesota          MN  severe weather           3060.0      680.167139   
3  Minnesota          MN  severe weather           3000.0      680.167139   
4  Minnesota          MN  severe weather           2550.0      680.167139   
5  Minnesota          MN  severe weather           1740.0      250.000000   
6  Minnesota          MN  severe weather           1860.0      680.167139   

   CUSTOMERS.AFFECTED  RES.PRICE  COM.PRICE  IND.PRICE  RES.PERCEN  \
1             70000.0      11.60       9.18       6.81   35.549073   
3             70000.0      10.87       8.19       6.07   28.097672   
4             68200.0      11.79       9.25       6.71   31.994099   
5            250000.0      13.07      10.16       7.74   33.982576   
6             60000.0      10.63       8.34       6.15   31.192784   

   COM.PERCEN  IND.PERCEN  RES.MONEY.LOST  COM.MONEY.LOST  IND.MONEY.LOST  \
1   32.225029   32.202431        14304.

# TODO
## Baseline model
### Categorical features
1. Cause cateogory
2. NERC region
3. Sector
### Numerical features
1. Average electricity consumption (sales)
2. Duration
3. Price
4. Customers served


## Final model
### Categorical features
1. Cause cateogory
2. Date of time (Morning, Evening, Night)
3. Date (Workday or weekend)
4. Geographical region
5. NERC region
6. Sector

### Numerical features
1. Average electricity consumption (sales)
2. Duration
3. Price
4. Customers served

### Baseline Model

In [ ]:
# TODO

### Final Model

In [ ]:
# TODO

### Fairness Analysis

In [ ]:
# TODO